<p style="text-align:center">
PSY 394U <b>Data Analytics with Python</b>, Spring 2018


<img style="width: 400px; padding: 0px;" src="https://github.com/sathayas/JupyterAnalyticsSpring2018/blob/master/images/Title_pics.png?raw=true" alt="title pics"/>

</p>

<p style="text-align:center; font-size:40px; margin-bottom: 30px;"><b> Support vector machine (SVM) </b></p>

<p style="text-align:center; font-size:18px; margin-bottom: 32px;"><b>February 15, 2018</b></p>

<hr style="height:5px;border:none" />

# 1. What is a support vector machine?
<hr style="height:1px;border:none" />

## The line must be drawn here

Let's say, you have two clusters of data points that are separated from each other. 